In [ ]:
import tensorflow as tf
from tensorflow.python.util import nest

In [ ]:
def adapt_model(model, dataset):
    """Adapt the preprocessing layers, e.g. Normalization(), to the dataset.
    """
    if isinstance(dataset, tf.data.Dataset):
        x = dataset.map(lambda x, y: x)
    else:
        x = nest.flatten(dataset)
    
    def get_output_layer(tensor):
        tensor = nest.flatten(tensor)[0]
        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.InputLayer):
                continue
            input_node = nest.flatten(layer.input)[0]
            if input_node is tensor:
                return layer
        return None

    for index, input_node in enumerate(nest.flatten(model.input)):
        def get_data(*args):
            return args[index]

        if isinstance(x, tf.data.Dataset):
            temp_x = x.map(get_data)
        else:
            temp_x = x[index]
        layer = get_output_layer(input_node)
        while isinstance(layer, preprocessing.PreprocessingLayer):
            layer.adapt(temp_x)
            layer = get_output_layer(layer.output)
    return model